In [8]:
    try:
        import commonmark as CommonMark
        from commonmark import Parser
        from commonmark.render.renderer import Renderer
    except:
        import CommonMark
        from CommonMark import Parser
        from CommonMark.render.renderer import Renderer
    from textwrap import indent, dedent
    from importnb import Notebook
    from collections import UserList
    from abc import abstractmethod, ABCMeta
    try:
        from IPython.display import display, Markdown, HTML
        from IPython.core.inputtransformer import InputTransformer
        from IPython import get_ipython
    except:
        class InputTransformer:
            def __init__(self, *args, **kwargs): ...
            def transform_cell(self, str): return dedent(str)
            
        def get_ipython(): ...

## Markdown to code

Markdown is converted to code by persisting all code_blocks and replacing all non-code lines with blank lines.  The blank line replacement assures the right line numbering.

* Create a custom CommonMark renderer for code only.  It maintains the lines and positions of the original source to give better error messages.

* The commonmark renderer only catches code cells

    > There is an interesting discussion to be had about the role of inline cells.

Code fences can't be mixed with indented code at this point.

In [9]:
    class CodeRenderer(Renderer):
        def code(self, node, entering):
            if self.user_expressions is not None: 
                self.user_expressions += node.literal,
                
        def code_block(self, node, entering):
            print(99,entering)
            self.slices.append(
                slice(node.sourcepos[0][0]-1, node.sourcepos[1][0])
            )
            spaces = ' '*(node.sourcepos[0][1])

            while len(self.buf.splitlines()) <  node.sourcepos[0][0]-1:
                self.out("\n")
            if node.is_fenced:
                self.out(indent("{}{}\n".format('%%' if node.info else '', node.info), spaces))
            self.out(indent(node.literal, spaces))
            
        repr = staticmethod(Markdown)
            
        def __call__(self, str, *user_expressions):
            self.source = str.splitlines(True)
            self.slices = []
            self.user_expressions = user_expressions
            walker = Parser().parse(str)
            
            parsed = dedent(self.render(walker))
            stripped = parsed.lstrip()
            if stripped.startswith('%%'):
                parsed = stripped

            return parsed, self.user_expressions

    renderer = CodeRenderer()

In [10]:
    class CodeRenderer(Renderer):
        def code(self, node, entering):
            if self.user_expressions is not None: 
                self.user_expressions += node.literal,
                
        def code_block(self, node, entering):
            if node.is_fenced: return
            self.slices.append(slice(node.sourcepos[0][0]-1, node.sourcepos[1][0]))            
            
        repr = staticmethod(Markdown)
            
        def __call__(self, str, *user_expressions):
            str = ''.join(str)
            source = str.splitlines(True)
            self.slices = []
            self.user_expressions = user_expressions
            self.render(Parser().parse(str))
            parsed = dedent(combine_slices(str, self.slices))                
            return parsed, self.user_expressions

    renderer = CodeRenderer()

In [11]:
    import ast, IPython

In [12]:
    import textwrap

In [13]:
    def transform(str):
        str = ''.join(str)
        source = renderer(str)[0]  
        return source.splitlines(True)

In [14]:
    def load_ipython_extension(ip=None):
        ip = ip or get_ipython()
        if renderer not in ip.input_transformers_cleanup:
            ip.input_transformers_cleanup.insert(0, transform)

In [15]:
    def unload_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformers_cleanup = [
            object for object in ip.input_transformers_cleanup 
            if object != renderer
        ]
        

    renderer("\nTest")

In [16]:
    def requote(str, punc=''):
        str = ''.join(str)
        _ = '"""'
        
        if _ in str: _ = "'''"
        
        if not str.strip(): _ = punc = ''
        
        end = len(str.rstrip())
        str, end = str[:end], str[end:]
        
        if str and str.endswith(_[0]):
            str += ' '
                        
        return F"{_}{str}{_}{punc}{end}"

In [17]:
    def count_indent(str): return len(str) - len(str.lstrip())

In [33]:
    def combine_slices(str, slices, *code, **prior):
        str += '\n'
        slash = str.startswith('\\')
        lines = str.lstrip('\\').splitlines(True)
        min =0
        prior = {slice: slice(0, 0), 'tabs': 0}
        last_line = ''
        for range in slices:
            block = lines[range]
            for indented in block:
                if indented.strip(): 
                    tabs = count_indent(indented)
                    break
                    
            min = min or tabs

            if prior['tabs'] > tabs: tabs = prior['tabs']
            
            if prior['tabs'] == tabs and last_line.rstrip().endswith(':'):
                tabs = prior['tabs'] + 4
            
            code += tuple(indent(
                ' '*(tabs-min) + requote(lines[prior[slice].stop: range.start]), ' '*min
            ).splitlines(True))
                                    
            code += tuple(block)
            for last_line in reversed(code):
                if last_line.strip(): break
                    
            prior.update({'tabs': count_indent(last_line), slice: range})

        
        if last_line.rstrip().endswith(':'):
            tabs = prior['tabs'] + 4
        else:  tabs = 0
            
        rest = lines[prior[slice].stop:]
                
        punc = ''
        
        if not ''.join(rest).strip(): return ''.join(code)
        if slices: punc = ';'
        elif (
            not lines[0].strip() or str.rstrip().endswith(';')
        ): punc = ';'

        code += tuple(indent(
            ' '*(tabs) + requote(rest, punc=punc), ' '*min
        ).splitlines(True))

        code = ''.join(code)
        
        return code
    

In [20]:
    class MarkdownImporter(Notebook):
        extensions = '.md.ipynb',
        def format(self, str): 
            source, user_expressions = renderer(str)
            return super().format(source)

In [27]:
    try:
        from IPython.display import HTML, Javascript
        
        style = HTML("""<style>
        .code-cell-toggle {
            position: fixed;
            bottom: 20px;
            right: 20px;
        }
        .code-cell-hidden .code_cell .input {
            display: none;
        }
        </style>""")

        toggle = Javascript("""$("button.code-cell-toggle").remove();
        var btn = $("<button/>", {"class": "code-cell-toggle"}).text("</>");
        btn.bind("click", function(){
            $("body").toggleClass("code-cell-hidden");
        });
        $("#notebook").append(btn);
        if (!window.location.host.startsWith('localhost')){
            $("body").addClass("code-cell-hidden");
        };""")
    except: ...

In [28]:
    class Test(__import__('unittest').TestCase): 
        def setUp(Test):
            %reload_ext pidgin
            load_ipython_extension()
            from nbformat import write, v4
            with open('test_markdown.md.ipynb', 'w') as file:
                write(v4.new_notebook(cells=[v4.new_code_cell("""Some paragraph\n\n    a=42""")]), file)
                
        def runTest(Test):
            global test_markdown
            import test_markdown
            assert test_markdown.__file__.endswith('.ipynb')
            assert test_markdown.a is 42

In [29]:
    if __name__ == '__main__':
        %reload_ext pidgin

AttributeError: 'DisplayStrings' object has no attribute 'previous'

### if __name__ == '__main__': 
        __import__('unittest').TextTestRunner().run(Test())